In [ ]:
import requests, os, datetime, json
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
username = # steam username
user_id = # Steam64 id (https://steamid.xyz/)

## Grab Total Playtime

In [ ]:
r = requests.get("https://steamcommunity.com/id/{}/games/?tab=all".format(username))
r.raise_for_status()
soup_games = BeautifulSoup(r.text, "html.parser")

In [ ]:
data = {}

for game in soup_games.find_all('script', attrs={'language':'javascript'})[0].contents[0].split('appid'):
    if 'TF2' not in game:
        continue
    data['total_time'] = game.split('hours_forever":')[1].split(',"')[0].replace('"', "")
    break
data

## Grab Player Stats

In [ ]:
r = requests.get("https://steamcommunity.com/id/{}/stats/TF2?tab=stats".format(username))
r.raise_for_status()

soup = BeautifulSoup(r.text, "html.parser")
stuff = soup.find_all('div', attrs={'class':'classExtended'}) # get records for 9 classes

In [ ]:
hrs = soup.text.split()
cs = soup.find_all('div', class_="className")
j = 0
classes = {}

for i, s in enumerate(hrs):
    if 'hrs' in s:
        classes[cs[j].text] = {'hours':hrs[i-1]}
        j += 1
        
classes

In [ ]:
for i, info in enumerate(stuff):
    for stat in info.find_all('div', class_='personalRecordStat'):
        work = stat.text.split()
        num = work[-1]
        stat_name = " ".join(work[:-1])
        
        classes[cs[i].text][stat_name] = num     
        
classes;

In [ ]:
data['achievements'] = soup.find_all('div', class_='achievementStatusText')[0].text.split(':')[1].strip()

for row in soup.find_all('div', class_='summaryStatRow'):
    data[row.text.split(':')[0].strip()] = " ".join(row.text.split(":")[1:]).strip()

for row in soup.find('div', attrs={'id':'personalRecords'}).find_all('div', class_='personalRecordStat'):
    data[row.find('div', class_='recordStatLabel').text] = row.find('span').text

hrsl = str(soup.find('div', attrs={'id':'playtimeBody'})).split('<br/>')
data['playtime_two_weeks'] = hrsl[1]
data['playtime_on_record'] = hrsl[2].split("\t")[0]
data;

## Grab Inventory

In [ ]:
r = requests.get("http://steamcommunity.com/inventory/{}/440/2?l=english&count=5000".format(user_id))
r.raise_for_status()
soup_inventory = BeautifulSoup(r.text, "html.parser")

In [ ]:
tf2_inventory = json.loads(soup_inventory.text)
tf2_inventory.keys()

In [ ]:
keys = ['name', 'type', 'market_name', 'tags', 'classid', 'instanceid']
strange_items = []

for item in tf2_inventory['descriptions']:
    if item['tags'][0]['internal_name'] == 'strange':
        print(item['type'], item['name'])

        strange_item = {}
        for key in keys:
            try:
                strange_item[key] = item[key]
            except:
                pass
        
        # filter descriptions list
        try:
            new_descriptions = []
            for value in item['descriptions']:
                for test in ['(', 'Killstreak', 'Sheen', 'Halloween']:
                    if test in value['value']:
                        new_descriptions.append(value['value'])
                        break

            strange_item['descriptions'] = new_descriptions
        except:
            pass
        
        strange_items.append(strange_item)

## Save Data

In [ ]:
# Get current date
today = datetime.datetime.now()
new_folder = str(today.month)+"_"+str(today.day)+"_"+str(today.year)
path = os.path.join('data',  new_folder)

# Make new folders
os.makedirs(path)
os.makedirs(os.path.join(path, 'html'))

In [ ]:
with open(os.path.join(path,'general_data.json'), 'w') as f:
    json.dump(data, f)
    
with open(os.path.join(path,'class_stats.json'), 'w') as f:
    json.dump(classes, f)

with open(os.path.join(path,'strange_items.json'), 'w') as f:
    json.dump(strange_items, f)

with open(os.path.join(path, 'html', 'stats.html'), 'w', encoding='utf-8') as f:
    f.write(str(soup))
    
with open(os.path.join(path, 'html', 'inventory.html'), 'w', encoding='utf-8') as f:
    f.write(str(soup_inventory))

with open(os.path.join(path, 'html', 'games.html'), 'w', encoding='utf-8') as f:
    f.write(str(soup_games))